In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

# Intersection over Union
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


In [24]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "5"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope

In [25]:
# Global Height and Width
H = 512
W = 512

#  Creating a directory 
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)



In [26]:
# Seeding
np.random.seed(42)
tf.random.set_seed(42)

# Directory for storing files
create_dir("/content/drive/MyDrive/Mined/remove_bg")

# Loading model: DeepLabV3+
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
    model = tf.keras.models.load_model("/content/drive/MyDrive/Mined/DeepLabV3+.h5")

# Here check model summary
model.summary()

# Load the dataset
data = glob("/content/drive/MyDrive/Mined/test/*")

for path in tqdm(data, total=len(data)):

      # Extracting Image name
      name = path.split("/")[-1].split(".")[0]
  
      # Read the image using OpenCV
      image = cv2.imread(path, cv2.IMREAD_COLOR)
      h, w, _ = image.shape

      # Resize with global height and width
      x = cv2.resize(image, (W, H))
      x = x/255.0
      x = x.astype(np.float32)
      x = np.expand_dims(x, axis=0)
      

      # Prediction from model DeepLabV3+ model
      y = model.predict(x)[0]
      y = cv2.resize(y, (w, h))
      y = np.expand_dims(y, axis=-1)
      y = y > 0.5

      photo_mask = y
      bg_mask = np.abs(1-y)

      masked_photo = image * photo_mask
      bg_mask = np.concatenate([bg_mask, bg_mask, bg_mask], axis=-1)
      bg_mask = bg_mask * [0, 0, 0] # Gave Background to white
      photo = masked_photo + bg_mask
      cv2.imwrite(f"/content/drive/MyDrive/Mined/remove_bg/{name}.png", photo)
    

Model: "functional_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                       

100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


In [1]:
!pip list

Package                       Version
----------------------------- ---------------------
absl-py                       1.0.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.11.4
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.1.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.4
branca                        0.4.2
bs4                           0.0.1
CacheC